In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
path = "proj1/fold{}/{}.csv"

trains = []
tests = []
for fold in range(1,11):
    train = pd.read_csv(path.format(fold, "train"))
    test = pd.read_csv(path.format(fold, "test"))
    test_y = pd.read_csv(path.format(fold, "test_y"))
    trains.append(train)
    tests.append(test)

all_train = pd.concat(trains)
all_test = pd.concat(tests)


all_train['Total_SF'] = all_train[['First_Flr_SF','Second_Flr_SF','Total_Bsmt_SF']].copy().sum(axis=1)
all_train['Total_Bath'] = all_train.loc[:,'Full_Bath'] + (0.5 * all_train.loc[:,'Half_Bath'])
all_train['Has_Pool'] = all_train['Pool_QC'].map({"No_Pool":0, 'Excellent':1, 'Good':1, 'Typical':0, 'Fair':0})
all_train['Has_Quality_Kitchen'] = all_train['Kitchen_Qual'].map({'Excellent':1, 'Good':1, 'Typical':0, 'Fair':0, 'Poor':0})
all_train['Has_Quality_Basement'] = all_train['Bsmt_Qual'].map({'Excellent':1, 'Good':1, 'Typical':0, 'Fair':0, 'Poor':0})

In [39]:
# delete these numeric columns
corrs = all_train.select_dtypes(include='number').corr().Sale_Price.abs().sort_values()
corrs[corrs < 0.1].index.values

array(['BsmtFin_SF_2', 'Misc_Val', 'Mo_Sold', 'Three_season_porch',
       'Year_Sold', 'Bsmt_Half_Bath', 'Low_Qual_Fin_SF', 'Pool_Area'],
      dtype=object)

In [40]:
corrs

BsmtFin_SF_2            0.008351
Misc_Val                0.018703
Mo_Sold                 0.022928
Three_season_porch      0.028741
Year_Sold               0.030741
Bsmt_Half_Bath          0.039697
Low_Qual_Fin_SF         0.040053
Pool_Area               0.062610
Has_Pool                0.111545
Kitchen_AbvGr           0.121616
Screen_Porch            0.126030
Enclosed_Porch          0.136201
BsmtFin_SF_1            0.142277
Bedroom_AbvGr           0.148072
Bsmt_Unf_SF             0.176934
Lot_Frontage            0.209532
Longitude               0.248249
PID                     0.252680
Second_Flr_SF           0.262596
Lot_Area                0.283330
Half_Bath               0.283937
Bsmt_Full_Bath          0.285094
Latitude                0.296598
Open_Porch_SF           0.317107
Wood_Deck_SF            0.334206
Fireplaces              0.480482
TotRms_AbvGrd           0.503554
Mas_Vnr_Area            0.524119
Garage_Yr_Blt           0.524968
Year_Remod_Add          0.531350
Has_Qualit

In [5]:
categorical = all_train.select_dtypes(include=['object', 'category'])
onehot = pd.get_dummies(categorical, prefix=categorical.columns)
onehot['Sale_Price'] = all_train.Sale_Price

In [6]:
corrs = onehot.corr().Sale_Price.abs().sort_values()

In [7]:
corrs[corrs > 0.3]

Mas_Vnr_Type_Stone                      0.309670
Exterior_2nd_VinylSd                    0.327811
Exterior_1st_VinylSd                    0.330611
Heating_QC_Typical                      0.336206
Foundation_CBlock                       0.345149
Sale_Condition_Partial                  0.353224
Overall_Qual_Average                    0.354704
Garage_Type_Attchd                      0.355065
MS_SubClass_Two_Story_1946_and_Newer    0.356001
Sale_Type_New                           0.357989
Overall_Cond_Average                    0.360200
Bsmt_Exposure_Gd                        0.364271
Garage_Type_Detchd                      0.368333
Overall_Qual_Very_Excellent             0.374093
Fireplace_Qu_Good                       0.374534
Overall_Qual_Very_Good                  0.404480
Garage_Finish_Fin                       0.419911
Garage_Finish_Unf                       0.427022
Exter_Qual_Good                         0.432994
Neighborhood_Northridge_Heights         0.438274
Bsmt_Qual_Typical   

In [28]:
[col for col in all_train.columns if 'kitchen' in col.lower()]

['Kitchen_AbvGr', 'Kitchen_Qual']

In [29]:
all_train['Kitchen_Qual'].unique()

array(['Typical', 'Good', 'Excellent', 'Fair', 'Poor'], dtype=object)

In [37]:
corrs[corrs.index.str.contains('Bsmt_Qual')]

Bsmt_Qual_Poor           0.033041
Bsmt_Qual_No_Basement    0.151477
Bsmt_Qual_Fair           0.160803
Bsmt_Qual_Good           0.209928
Bsmt_Qual_Typical        0.451294
Bsmt_Qual_Excellent      0.604159
Name: Sale_Price, dtype: float64